In [2]:
from skimage import data
import napari
import matplotlib.pyplot as plt
import skimage.io
from skimage import io
from ctypes import *
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tkinter import *
from PIL import ImageTk, Image
import cv2
from tifffile import imsave
import os
import time
from pypylon import pylon
import time, os, fnmatch, shutil
import numpy as np
from pypylon import genicam
import pyfirmata
from tkinter import messagebox
from tkinter import ttk
import tkinter as tk
from tkinter.messagebox import *
import h5py
from threading import Thread
from pyfirmata import Arduino, util
from tkinter import filedialog

#define camera functions   
def Exposure():
    # Print the model name of the camera.
    #print("Using device ", camera.GetDeviceInfo().GetModelName())
    a=float(scale3.get())
    camera.MaxNumBuffer = 2
    try:
        camera.Gain = camera.Gain.Max
    
    except genicam.LogicalErrorException:
         camera.GainRaw = camera.GainRaw.Max
    camera.Width = camera.Width.Max
    camera.Height = camera.Height.Max
    # camera.ExposureTime = camera.ExposureTime.Min
    camera.PixelFormat = "Mono12"
    camera.ExposureTime.SetValue(a)
    II=Snap_BF()
    img2 = Image.fromarray(cv2.resize(II/200, dsize=(600,400), interpolation=cv2.INTER_CUBIC))
    img1 = ImageTk.PhotoImage(image=img2)
    lmain1 = Label(frame1,image=img1,height=400,width=600)
    lmain1.imgtk = img1    
    lmain1.configure(image=img1) 
    lmain1.grid(row=0,column=0,rowspan=4,columnspan=4)
    
def Exposure1():
    # Print the model name of the camera.
    #print("Using device ", camera.GetDeviceInfo().GetModelName())
    a=float(entry_exposurefluo.get())
    camera.MaxNumBuffer = 2
    try:
        camera.Gain = camera.Gain.Max
    
    except genicam.LogicalErrorException:
         camera.GainRaw = camera.GainRaw.Max
    camera.Width = camera.Width.Max
    camera.Height = camera.Height.Max
    # camera.ExposureTime = camera.ExposureTime.Min
    camera.PixelFormat = "Mono12"
    camera.ExposureTime.SetValue(a)
    
def Exposure2():
    # Print the model name of the camera.
    #print("Using device ", camera.GetDeviceInfo().GetModelName())
    a=float(entry_exposurebf.get())
    camera.MaxNumBuffer = 2
    try:
        camera.Gain = camera.Gain.Max
    
    except genicam.LogicalErrorException:
         camera.GainRaw = camera.GainRaw.Max
    camera.Width = camera.Width.Max
    camera.Height = camera.Height.Max
    # camera.ExposureTime = camera.ExposureTime.Min
    camera.PixelFormat = "Mono12"
    camera.ExposureTime.SetValue(a)
    
def SetIllumination2_BIS():    
    a=scale2.get()
    percent2=a/100
    sortie2.write(percent2)   
    II=Snap_BF()
    img2 = Image.fromarray(cv2.resize(II/200, dsize=(600,400), interpolation=cv2.INTER_CUBIC))
    img1 = ImageTk.PhotoImage(image=img2)
    lmain1 = Label(frame1,image=img1,height=400,width=600)
    lmain1.imgtk = img1    
    lmain1.configure(image=img1) 
    lmain1.grid(row=0,column=0,rowspan=4,columnspan=4)
        
def SetIllumination1_BIS():    
    a=scale1.get()
    percent2=a/100
    sortie1.write(percent2)   
    II=Snap_BF()
    img2 = Image.fromarray(cv2.resize(II/200, dsize=(600,400), interpolation=cv2.INTER_CUBIC))
    img1 = ImageTk.PhotoImage(image=img2)
    lmain1 = Label(frame1,image=img1,height=400,width=600)
    lmain1.imgtk = img1    
    lmain1.configure(image=img1) 
    lmain1.grid(row=0,column=0,rowspan=4,columnspan=4)       

def SetIllumination2_BISa(a):    #bf
    percent2=a/100
    sortie2.write(percent2)
        
def SetIllumination1_BISa(a):   #fluo 
    percent2=a/100
    sortie1.write(percent2) 
    
def Snap_BF():
    camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly) 
    converter = pylon.ImageFormatConverter()
# converting to opencv bgr format
    converter.OutputPixelFormat = pylon.PixelType_Mono16
    converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
    grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)    
        # Access the image data
    image = converter.Convert(grabResult)
    img = image.GetArray()        
# Releasing the resource    
    camera.StopGrabbing()
    return img    

def Preview():
    camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly) 
    converter = pylon.ImageFormatConverter()

    converter.OutputPixelFormat = pylon.PixelType_Mono16
    converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned

    while camera.IsGrabbing():
        grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

        if grabResult.GrabSucceeded():
            image = converter.Convert(grabResult)
            img = image.GetArray()
            cv2.namedWindow('Preview', cv2.WINDOW_NORMAL)
            cv2.imshow('Preview', img)
            k = cv2.waitKey(1)
            if k == 27:
                break
        grabResult.Release()
       
    camera.StopGrabbing()
    cv2.destroyAllWindows()
    

def Video():
    k = int(scale4.get())
    camera.StopGrabbing()
    camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly) 
    converter = pylon.ImageFormatConverter()
    converter.OutputPixelFormat = pylon.PixelType_Mono16
    converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
    vivi=np.zeros([k,915,1371],'uint16')
    file_name1 = entry_filenamevideo.get()
    file_path1 = entry_directoryvideo.get()
    t2 = time.localtime()
    a=float(scale3.get())
    d = camera.ResultingFrameRate.GetValue()
    timestamp2 = time.strftime('%Y%m%d%H%M', t2)
    os.mkdir(file_path1 + '\\' + file_name1)
    f = open(file_path1 + '\\' + file_name1 + "\\meta_data.txt", "w")
    f.write("File name : " + file_name1 + "\nFile extension : .tif\nDate : " + timestamp2 + "\nImage size : 1371x915\nPixel deph : 16-bit\nExposure : " + str(a) + "ms\nFPS : " + str(d) + "\nPixel = 1.59um")
    f.close()
    for ii in range(k):
        grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

        if grabResult.GrabSucceeded():
            image = converter.Convert(grabResult)
            img = image.GetArray()
            vivi[ii,:,:]=(cv2.resize(img, dsize=(1371,915), interpolation=cv2.INTER_CUBIC))
        grabResult.Release()       
    camera.StopGrabbing()
    file_name1 = entry_filenamevideo.get()
    file_path1 = entry_directoryvideo.get()
    total=os.path.join(file_path1 + '\\' + file_name1, file_name1 +'.tif')
    imsave(total, vivi)
    

def Save_snapshot():
    II=Snap_BF()
    img1 = Image.fromarray(cv2.resize(II/100, dsize=(600,400), interpolation=cv2.INTER_CUBIC))
    img1=np.asanyarray(img1)
    file_name = entry_filename.get()
    file_path = entry_directory.get()
    suffix = '.tif'
    total=os.path.join(file_path, file_name +'1' + suffix)
    imsave(total,img1)
    
def Imagetime():
    a=int(entry_timeperhour.get())
    a1 = 60 * a
    n = int(entry_time.get())
    t_end = time.time() + 3600 * n
    if (var2.get() == 1) & (var1.get() == 0):
        w = ((n * 3600)/(a * 60))
        i = (w * 93245) * 10**(-6)
    elif (var2.get() == 0) & (var1.get() == 1):
        w = ((n * 3600)/(a * 60))
        i = (w * 93245) * 10**(-6)
    elif (var2.get() == 1) & (var1.get() == 1):
        w = ((n * 3600)/(a * 60)) * 2
        i = (w * 93245) * 10**(-6)
    barpro = 2/w
    expfluo=float(entry_exposurefluo.get())
    expbf=float(entry_exposurebf.get())
    rep = askokcancel ("Incubascope - Alert", "You will take " + str(w) + " pictures for a total of " + str(i) + " GB. Would you like to continue")
    if rep == 1:
        t2 = time.localtime()
        timestamp2 = time.strftime('%Y%m%d%H%M', t2)
        file_name1 = entry_filename1.get()
        file_path1 = entry_directory1.get()
        
        os.mkdir(file_path1 + '\\' + file_name1)
        os.mkdir(file_path1 + '\\' + file_name1 + '\\BF')
        os.mkdir(file_path1 + '\\' + file_name1 + '\\FLUO')
        f = open(file_path1 + '\\' + file_name1 + "\\meta_data.txt", "w")
        f.write("File name : " + file_name1 + "\nFile extension : .tif\nStart date : " + timestamp2 + "\nImage size : 5472x3672\nPixel deph : 32-bit\nExposure Fluo : " + str(expfluo) + "ms" + "\nExposure BF : " + str(expbf) + "ms" + "\nTime step : " + str(a1) + " min\nPixel = 1.59um")
        f.close()
        while time.time() < t_end:

            SetIllumination1_BISa(100)
            SetIllumination2_BISa(0)
            Exposure1()
            time.sleep(2)
            II1=Snap_BF()
            SetIllumination1_BISa(0)
            SetIllumination2_BISa(100)
            Exposure2()
            time.sleep(2)
            II2=Snap_BF()
            time.sleep(0.2)    
            SetIllumination2_BISa(0)            
            t = time.localtime()
            timestamp = time.strftime('%Y%m%d%H%M_', t)
            img1 = Image.fromarray(cv2.resize(II1, dsize=(5472,3672), interpolation=cv2.INTER_CUBIC))
            img1=np.asanyarray(img1)
            file_name1 = entry_filename1.get()
            file_path1 = entry_directory1.get()
            suffix = '.tif'
            print(var1.get())
            print(var2.get())
            if var2.get() == 1:
                total=os.path.join(file_path1 + '\\' + file_name1 + '\\FLUO', timestamp + file_name1 + '-Fluo ' + suffix)
                imsave(total,img1)
                print('test')
            img2 = Image.fromarray(cv2.resize(II2, dsize=(5472,3672), interpolation=cv2.INTER_CUBIC))
            img2=np.asanyarray(img2)
            file_name1 = entry_filename1.get()
            file_path1 = entry_directory1.get()
            if var1.get() == 1:
                total1=os.path.join(file_path1 + '\\' + file_name1 + '\\BF', timestamp + file_name1 + '-BF ' + suffix)
                imsave(total1,img2)
                print('test')
            print(t_end)
            my_progress['value'] += (barpro)*100
            time.sleep(a1)
        t1 = time.localtime()
        timestamp1 = time.strftime('%Y%m%d%H%M', t1)
        f = open(file_path1 + "\\meta_data.txt", "w")
        f.write("File name : " + file_name1 + "\nFile extension : .tif\nStart date : " + timestamp2 + "\nEnd date : " + timestamp1 + "\nImage size : 5472x3672\nPixel deph : 32-bit\nExposure Fluo : " + str(expfluo) + "ms" + "\nExposure BF : " + str(expbf) + "ms" + "\nTime step : " + str(a1) + " min\nPixel = 1.59um")
        f.close()
        my_progress.stop()
        messagebox.showinfo("Incubascope - Alert", "Acquisition completed")
    
        
def Start():
    mon_thread.start()
    
def Start1():
    mon_thread1.start()

def opennapari():
    filename = filedialog.askopenfilename(filetypes=[("TIF files", ".tif")])
    im = io.imread(filename)
    if cmb.get() == "Full":
        k = 1
    elif cmb.get() == "2":
        k = 2
    elif cmb.get() == "3":
        k = 3
    elif cmb.get() == "4":
        k = 4
    elif cmb.get() == "10":
        k = 10
    plt.imshow(im[:,:,0])
    with h5py.File('random3.hdf5', 'w') as f:
        dset = f.create_dataset("default", data=im)
    with h5py.File('random3.hdf5', 'r') as f:
        data_set = f['default']
        data = data_set[:,::k,::k]
    data.shape
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(data)

def main():
    global camera
    global mon_thread1
    global mon_thread
    global scale1
    global scale2
    global scale3
    global scale4
    global sortie1
    global sortie2
    global frame0
    global frame1
    global entry_filename
    global entry_directoryvideo
    global entry_filename1
    global entry_filenamevideo
    global entry_directory
    global entry_directory1
    global entry_timeperhour
    global entry_time
    global entry_exposurefluo
    global entry_exposurebf
    global my_progress
    global var1, var2
    global cmb

    #camera initialization
    camera = pylon.InstantCamera(
    pylon.TlFactory.GetInstance().CreateFirstDevice())

    camera.Open()

        # Print the model name of the camera.
    camera.MaxNumBuffer = 2

    camera.Width = camera.Width.Max
    camera.Height = camera.Height.Max
        # camera.ExposureTime = camera.ExposureTime.Min
    camera.PixelFormat = "Mono12"
    camera.ExposureTime.SetValue(20)

    #Arduino initialization
    carte = Arduino('COM3')
    sortie1 = carte.get_pin('d:3:p')
    sortie2 = carte.get_pin('d:6:p')


    root = Tk()
    root.resizable(True, True)
    root.title('Incubascope - Software')
    root.geometry("1450x750")
    root.configure(background='white')

    notebook = ttk.Notebook(root)
    notebook.grid(row=1, column=0)
    frame11 = Frame(root, width=1450, height=775, background="white")
    frame22 = Frame(root, width=1450, height=775, background="white")

    v = DoubleVar()  
    y = DoubleVar()
    f = DoubleVar()
    p = DoubleVar()
    var1 = IntVar()
    var2 = IntVar()
    II = Snap_BF()

    mon_thread=Thread(target=Imagetime)
    mon_thread1=Thread(target=Preview)
        
    frame0 = Frame(root, width=1500, height=80, background="white")
    Title=Label(frame0, text='INCUBASCOPE - Acquisition software', background="white")
    Title.config(font=('Arial', 18))
    Title.grid(column=0, row=0, rowspan=1, columnspan=1)
    test0 = Image.open('C:\\Users\\BiOf\\img.jpg')
    test0=test0.resize((340, 120), Image.ANTIALIAS)
    photo0 = ImageTk.PhotoImage(test0)
    label0 = Label(frame0,image=photo0,height=120,width=340)
    label0.image = photo0
    label0.grid(column=1,row=0,columnspan=3)
    frame0.grid(row=0, column=0, rowspan=1, columnspan=6)


    frame1 = Frame(frame11, width=1300, height=800, background="grey")
    img2 = Image.fromarray(cv2.resize(II/200, dsize=(600,400), interpolation=cv2.INTER_CUBIC))
    img1 = ImageTk.PhotoImage(image=img2)
    lmain1 = Label(frame1,image=img1,height=400,width=600)
    lmain1.imgtk = img1    
    lmain1.configure(image=img1) 
    lmain1.grid(row=0,column=0,rowspan=4,columnspan=4)

    button_start1 = Button(frame1, text='Start preview', bg='Seagreen1', height=2, width=20, command = Preview)
    button_start1.grid(row=0, column=6, columnspan=3)

    scale1 = Scale( frame1, variable = v, from_ = 0, to = 100, orient = VERTICAL, resolution=20, tickinterval=20, length=150,
        label='Fluo')
    scale1.grid(row=2, column=6, columnspan=2)
    scale2 = Scale( frame1, variable = y, from_ = 0, to = 100, orient = VERTICAL, resolution=20, tickinterval=20, length=150,
        label='BF')
    scale2.grid(row=3, column=6, columnspan=2)
    scale3 = Spinbox(frame1, from_=20, to=100000)
    scale3.grid(row=5, column=1, columnspan=2)

    text_exposure = Label(frame1, text='Exposure', height=2, width=10,background="white")
    text_exposure.grid(row=5, column=0, columnspan=2)
    text_imagevideo = Label(frame1, text='nbr image', height=2, width=10,background="white")
    text_imagevideo.grid(row=6, column=0, columnspan=2)
    text_imagevideopath = Label(frame1, text='Directory path', height=2, width=10,background="white")
    text_imagevideopath.grid(row=7, column=0, columnspan=2)
    text_imagevideoname = Label(frame1, text='File name', height=2, width=10,background="white")
    text_imagevideoname.grid(row=8, column=0, columnspan=2)

    scale4 = Spinbox(frame1, from_=1, to=100000)
    scale4.grid(row=6, column=1, columnspan=2)
    button_video = Button(frame1, text='Start video',bg='Seagreen2', height=2, width=10, command = Video)
    button_video.grid(row=6, column=2, columnspan=5)
    entry_directoryvideo = Entry(frame1)
    entry_directoryvideo.insert(10,"C:/Users/Admin")
    entry_directoryvideo.grid(row=7, column=1, columnspan=2)
    entry_filenamevideo = Entry(frame1)
    entry_filenamevideo.insert(10,"default")
    entry_filenamevideo.grid(row=8, column=1, columnspan=2)

    button_illumination2 = Button(frame1, text='Confirm',bg='light blue', height=1, width=10, command = SetIllumination2_BIS)
    button_illumination2.grid(row=3, column=8)
    button_illumination1 = Button(frame1, text='Confirm',bg='light blue', height=1, width=10, command = SetIllumination1_BIS)
    button_illumination1.grid(row=2, column=8)
    button_exposure = Button(frame1, text='Confirm',bg='light blue', height=1, width=10, command = Exposure)
    button_exposure.grid(row=5, column=2, columnspan=5)


    frame1.grid(row=2, column=0, rowspan= 1, columnspan=1)

    frame2 = Frame(frame11, width=500, height=400, background="grey")
    text_directory = Label(frame2, text='Directory path', height=2, width=10,background="white")
    text_directory.grid(row=1, column=0)
    text_filename = Label(frame2, text='File name', height=2, width=10,background="white")
    text_filename.grid(row=2, column=0)
    text_time1 = Label(frame2, text='Image gap(min)', height=2, width=15,background="white")
    text_time1.grid(row=3, column=0)
    text_time2 = Label(frame2, text='Time span(hour)', height=2, width=15,background="white")
    text_time2.grid(row=4, column=0)
    text_time2 = Label(frame2, text='Exposure Fluo', height=2, width=10,background="white")
    text_time2.grid(row=5, column=0)
    text_time2 = Label(frame2, text='Exposure BF', height=2, width=10,background="white")
    text_time2.grid(row=6, column=0)

    button_snap = Button(frame2, text='Save snapshot',bg='Seagreen2', height=2, width=10, command = Save_snapshot)
    button_snap.grid(row=0, column=2,columnspan=1)

    entry_directory = Entry(frame2)
    entry_directory.insert(10,"C:/Users/Admin")
    entry_directory.grid(row=1, column=2)
    entry_filename = Entry(frame2)
    entry_filename.insert(10,"default")
    entry_filename.grid(row=2, column=2)
    entry_directory1 = Entry(frame2)
    entry_directory1.insert(10,"C:/Users/Admin")
    entry_directory1.grid(row=1, column=3)
    entry_filename1 = Entry(frame2)
    entry_filename1.insert(10,"default")
    entry_filename1.grid(row=2, column=3)


    entry_timeperhour = Spinbox(frame2, from_=1, to=100000)
    entry_timeperhour.grid(row=3, column=3)
    entry_time = Spinbox(frame2, from_=1, to=100000)
    entry_time.grid(row=4, column=3)
    entry_exposurefluo = Spinbox(frame2, from_=10000, to=100000)
    entry_exposurefluo.grid(row=5, column=3)
    entry_exposurebf = Spinbox(frame2, from_=20, to=100000)
    entry_exposurebf.grid(row=6, column=3)

    button_snap1 = Button(frame2, text='Timelapse',bg='Seagreen2', height=2, width=10, command = Start)
    button_snap1.grid(row=0, column=3,columnspan=1)
    #button_snap2 = Button(frame2, text='Stop Timelapse',bg='sSeagreen2', height=2, width=10, command = Start)
    #button_snap2.grid(row=0, column=4,columnspan=1)

    my_progress = ttk.Progressbar(frame2, orient=HORIZONTAL, length=150, mode = "determinate")
    my_progress.grid(row=8, column=3)

    c1 = Checkbutton (frame2, text = 'BF     ', variable = var1, onvalue=1, offvalue=0)
    c1.grid(row=0, column=4)
    c2 = Checkbutton (frame2, text = 'FLUO', variable = var2, onvalue=1, offvalue=0)
    c2.grid(row=1, column=4)

    frame2.grid(row=2, column=1, rowspan= 8, columnspan=4)
    frame11.grid(row=0, column=0, columnspan=8)

    selection = ('Full', '2', '3', '4', '10')
    selected = tk.StringVar()
    cmb = ttk.Combobox(frame22, textvariable=selected)
    cmb['values'] = selection
    cmb.current(0)
    cmb.grid(row=2, column=1)
    button_napari = Button(frame22, text='Open preview',bg='light blue', height=1, width=10, command = opennapari)
    button_napari.grid(row=1, column=1)

    frame22.grid(row=0, column=0)

    notebook.add(frame11, text='Software')
    notebook.add(frame22, text='Analyze')
    

    root.mainloop()


if __name__ == "__main__":
    main()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\programdata\miniconda3\lib\tkinter\__init__.py", line 1884, in __call__
    return self.func(*args)
  File "C:\Users\BiOf\AppData\Local\Temp/ipykernel_11384/4287200676.py", line 101, in SetIllumination1_BIS
    sortie1.write(percent2)
  File "c:\programdata\miniconda3\lib\site-packages\pyfirmata\pyfirmata.py", line 545, in write
    self.board.sp.write(msg)
  File "c:\programdata\miniconda3\lib\site-packages\serial\serialwin32.py", line 317, in write
    raise SerialException("WriteFile failed ({!r})".format(ctypes.WinError()))
serial.serialutil.SerialException: WriteFile failed (PermissionError(13, 'The device does not recognize the command.', None, 22))
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\programdata\miniconda3\lib\tkinter\__init__.py", line 1884, in __call__
    return self.func(*args)
  File "C:\Users\BiOf\AppData\Local\Temp/ipykernel_11384/4287200676.py", line 101, i